In [1]:
!pip install -q grobid-client langchain openai faiss-cpu PyPDF2 tiktoken chromadb

In [7]:
PROJECT_ID = "jerry-argolis"  # @param {type:"string"}
LOCATION = "us-central1" # @param {type:"string"}
FOLDER_ID = "1UHgK76I5-FVr8r0KRvKCWo8S9zX41TAD" # @param {type:"string"}

In [3]:
from platform import python_version
print(python_version())

3.10.12


In [4]:
# Install Vertex AI LLM SDK, langchain and dependencies
# Remember to click RESTART RUNTIME after this step.
! pip install config --upgrade --user
! pip install google-cloud-aiplatform google-api-python-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.3 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.1
    Uninstalling shapely-2.0.1:
      Successfully uninstalled shapely-2.0.1


In [4]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [5]:
import langchain
import vertexai

# Vertex AI
from google.cloud import aiplatform
from langchain.llms import VertexAI

vertexai.init(project=PROJECT_ID, location=LOCATION)

print(f"LangChain version: {langchain.__version__}")
print(f"Vertex AI SDK version: {aiplatform.__version__}")

LangChain version: 0.0.278
Vertex AI SDK version: 1.31.1


In [8]:
from langchain.document_loaders import GoogleDriveLoader

loader = GoogleDriveLoader(
    folder_id=FOLDER_ID,
    recursive=False
)
docs = loader.load()

In [9]:
# split the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=200)
docs = text_splitter.split_documents(docs)

print(f"# of chunks = {len(docs)}")

# of chunks = 224


In [10]:
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import Chroma

# Embedding
embeddings = VertexAIEmbeddings()
db = Chroma.from_documents(docs, embeddings)

In [11]:
# Create chain to answer questions
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# LLM model
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0,
    top_p=0.95,
    top_k=40,
    verbose=True,
)

prompt_template = """
You are a trustworthy consultant.
Answer the users question based on the context only.
Do not make up data. If you cannot find answer in the context, then say 'Sorry, I cannot find the answers in the documents.'

{context}

Question: {question}
"""

PROMPT = PromptTemplate(
    template = prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

# Seting the approriate retriver
#retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})
retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.4, "k": 4})

qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
)

In [13]:
# query = "What is the best AI model to implemnet fraud detection system"
query = "fraud detection 시스템을 구축할 때 가장 적합한 AI 모델은?"
result = qa({"query": query})

print(f"Query: ", result["query"])
print(f"Answer: ", result["result"])
print(f"Source: ", result["source_documents"])

Query:  fraud detection 시스템을 구축할 때 가장 적합한 AI 모델은?
Answer:  The authors suggest that the most suitable AI model for fraud detection is the decision tree. This is because the decision tree is able to learn from the data and identify patterns that can be used to detect fraud. The authors also suggest that the decision tree should be normalized in order to improve its accuracy.
Source:  [Document(page_content='정보보호학회논문지 ( 2015. 2) 145\n또한, 더 많은 유형의 데이터가 학습될 것이고 의사\n결정나무는 좀 더 복잡해질 수 밖에 없다. 게다가 전자금융사기 방식은 하루가 다르게 발전하고 있기 때문\n에 좀 더 다양한 데이터와 실험을 통해서 탐지 방식을 \n발전시킬 필요성이 있다. 그리고 좀 더 효율적인 시스템을 구축 하려면 기계학습에 의해서 의사결정나무를 \n매일 혹은 정기적으로 재구성하여 좀 더 정교하게 발\n전시켜 나아가야 할 것이다.\n \nReferences\n[1] The Phishing Fraud Occurrence and \nD a m a g e  R e i m b u r s e m e n t  S t a t u s ,  t h e  Financial Supervisory Service, May. \n2014.\n[2] The Electronic Financial Transaction Act \nArticle 9, May. 2013.\n[3] Financial Security Comprehensive Plan, \nthe Financial Services Commission, Nov. 2013.\n[4] The Elect